# Compute the neon K-LL Auger spectrum

In [8]:
using JAC

In this tutorial, we wish to calculate the K-LL spectrum of atomic neon after $1s$ inner-shell ionization as it may occurs, for instance, following photon, electron or neutron impact. An autoionization of an atom or ion generally occurs if the initially-excited level is **energetically embedded into the continuum of the next higher charge state**, and then often also dominates the decay of the atom.

Since the neon K-shell ($1s$) electron has a binding energy larger than 800 eV, this condition is fulfilled and may, in priciple, even lead to a 4-5 times multiple ionization, although the initial excitation energy does not tell much of how many electrons will be emitted eventually. This depends of course on the particular decay path
of an atom and how easily further electrons can be *shaked-up* (or *off*) the atom during the de-excitation.


An Auger electron emission often leads to characteristic and energetically isolated lines as the kinetic energy of the emitted electrons are given simply by the difference of the (total) initial- and final-level energies. We therefore need to specify only the initial- and final-state multiplets as well as the process identifier `Auger` in order to calculate the K-LL spectrum with just the standard settings, but by defining a grid that is appropriate to describe *outgoing* electrons:

In [2]:
wa = Atomic.Computation("Neon K-LL Auger spectrum",  Nuclear.Model(10.); 
            grid=JAC.Radial.Grid("grid: by given parameters"; rnt = 2.0e-6, h = 5.0e-2, 
                                                              hp = 1.0e-2, NoPoints = 800), 
            initialConfigs=[Configuration("1s 2s^2 2p^6")],
            finalConfigs  =[Configuration("1s^2 2s^2 2p^4"), Configuration("1s^2 2s 2p^5"), 
                            Configuration("1s^2 2p^6")], process = JAC.Auger )
perform(wa)

Define a radial grid of type MeshGL with 806 grid points
 [rnt=2.000e-06, h=5.000e-02, hp=1.000e-02, NoPoints=800, r_max=5.098e+00;
  B-splines with break points at every 7th point, nsL=114, nsS=115, orderL=7, orderS=8, orderGL=7] 

... in perform['computation: SCF'] ...
perform-aa: Configuration: 1s_1/2^1 2s_1/2^2 2p_1/2^2 2p_3/2^4 
(Re-) Define a new standard subshell list.
(Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 10.0 with mass = 20.5, radius R = 2.8580081360657426 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      1s_1/2      -5.00667217e+01    -5.00667420e+01    +4.04942206e-07    
      2 

  1s_1/2::  en [a.u.] = -3.3913031e+01;   self-cons'cy = 8.4161e-04  [1.0980e-02 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -2.1206324e+00;   self-cons'cy = 5.0385e-03  [1.0980e-02 for sym-block kappa = -1]
  2p_1/2::  en [a.u.] = -1.3034423e+00;   self-cons'cy = 4.5935e-03  [3.9234e-02 for sym-block kappa = 1]
  2p_3/2::  en [a.u.] = -1.2940351e+00;   self-cons'cy = 8.7402e-04  [5.1961e-03 for sym-block kappa = -2]

Iteration 8 for symmetries ... 
  1s_1/2::  en [a.u.] = -3.3899208e+01;   self-cons'cy = 2.0385e-04  [2.4645e-03 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -2.1155039e+00;   self-cons'cy = 1.2107e-03  [2.4645e-03 for sym-block kappa = -1]
  2p_1/2::  en [a.u.] = -1.3021910e+00;   self-cons'cy = 4.8024e-04  [4.4827e-03 for sym-block kappa = 1]
  2p_3/2::  en [a.u.] = -1.2965235e+00;   self-cons'cy = 9.6053e-04  [7.3858e-03 for sym-block kappa = -2]

Iteration 9 for symmetries ... 
  1s_1/2::  en [a.u.] = -3.3898788e+01;   self-cons'cy = 6.1931e-06  [1.8785e-04

  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      2p_3/2      -1.25041630e+01    -1.25041630e+01    -5.91685043e-13    
      2      3p_3/2      -5.55802197e+00    -5.55802266e+00    +1.24792601e-07    
      3      4p_3/2      -3.12197027e+00    -3.12630099e+00    +1.38717688e-03    
      4      5p_3/2      -1.79261136e+00    -2.00074602e+00    +1.16106963e-01    
      5      6p_3/2      -2.43788145e-01    -1.38935143e+00    +4.69901146e+00    
      6      7p_3/2      +1.82172541e+00    -1.02071331e+00    +1.56030031e+00    
      7      8p_3/2      +4.36982343e+00    -7.81461386e-01    +1.17883134e+00    
      8      9p_3/2      +7.37525211e+00    -6.17436217e-01    +1.08371730e+00    
      :       :    
    109    110p_3/2      +1.00174076e+08    -4.13233009e-03    +1.00000000


AutoIonization.computeLines(): The computation of Auger rates and properties starts now ... 
------------------------------------------------------------------------------------------- 

Radial potential with effective charge Zbar=2.0001e+00 (Delta-Zbar=0.0000e+00).
Compute (Coulomb) Auger matrix of dimension 2 x 1 in the continuum- and initial-state bases for the transition [1- ...] and for partial wave d_5/2 ... done. 
Radial potential with effective charge Zbar=2.0001e+00 (Delta-Zbar=0.0000e+00).
Compute (Coulomb) Auger matrix of dimension 2 x 1 in the continuum- and initial-state bases for the transition [1- ...] and for partial wave d_3/2 ... done. 
Radial potential with effective charge Zbar=2.0001e+00 (Delta-Zbar=0.0000e+00).
Compute (Coulomb) Auger matrix of dimension 1 x 1 in the continuum- and initial-state bases for the transition [1- ...] and for partial wave s_1/2 ... done. 
Radial potential with effective charge Zbar=2.0001e+00 (Delta-Zbar=0.0000e+00).
Compute (Coulomb) 


This computation just applies simple *Bessel-type* continuum orbitals which are normalized on the basis of a pure sine behaviour at large radii. Moreover, the Auger amplitudes are computed by including the Coulomb interaction. Further control about this computation can be gained by re-defining the method for the integration of the continuum orbitals as well as by means of the `Auger.Settings`. For example, we can use a B-spline-Galerkin method and a normalization for a pure asymptotic Coulomb field to generate all continuum orbitals of interest:


In [3]:
setDefaults("method: continuum, Galerkin")
setDefaults("method: normalization, pure Coulomb")


Moreover, we can first have a look how the standard settings are defined at present: 


In [4]:
? AutoIonization.Settings

`struct  Settings`  ... defines a type for the details and parameters of computing Auger lines.

```
+ calcAnisotropy          ::Bool                         ... True, if the intrinsic alpha_2,4 angular parameters are to be calculated, 
                                                             and false otherwise.
+ printBeforeComputation  ::Bool                         ... True, if all energies and lines are printed before their evaluation.
+ selectLines             ::Bool                         ... True, if lines are selected individually for the computations.
+ selectedLines           ::Array{Tuple{Int64,Int64},1}  ... List of lines, given by tupels (inital-level, final-level).
+ minAugerEnergy          ::Float64                      ... Minimum energy of free (Auger) electrons to be included.
+ maxAugerEnergy          ::Float64                      ... Maximum energy of free (Auger) electrons to be included.
+ maxKappa                ::Int64                        ... Maximum kappa value of partial waves to be included.
+ operator                ::String                       ... Auger operator that is to be used for evaluating the Auger amplitudes: 
                                                             allowed values are: "Coulomb", "Breit", "Coulomb+Breit"
```

---

`AutoIonization.Settings()`  ... constructor for the default values of AutoIonization line computations


In [5]:
AutoIonization.Settings()

calcAnisotropy:                false  
printBeforeComputation:        false  
selectLines:                   false  
selectedLines:                 Tuple{Int64,Int64}[]  
minAugerEnergy:                0.0  
maxAugerEnergy:                1.0e6  
maxKappa:                      100  
operator:                      Coulomb  



Obviously, neither the Auger energies nor the symmetries of the (outgoing) partial wave are restricted. Apart from the Coulomb interaction, we could just included the Breit interaction (mainly for test purposes) or both, Coulomb+Breit together, and independent of whether this interaction has been incorporated in the SCF and CI computations before. In addition, we can also calculate the angular anisotropy coefficients as well as specify individual lines or symmetries to be included. We here leave the selection of individual lines to later and define the settings as:


In [6]:
autoSettings = AutoIonization.Settings(true, true, false, Tuple{Int64,Int64}[], 0.0, 1.0e5, 10, "Coulomb")

calcAnisotropy:                true  
printBeforeComputation:        true  
selectLines:                   false  
selectedLines:                 Tuple{Int64,Int64}[]  
minAugerEnergy:                0.0  
maxAugerEnergy:                100000.0  
maxKappa:                      10  
operator:                      Coulomb  



and re-run the computations with these settings:


In [7]:
wb = Atomic.Computation("Neon K-LL Auger spectrum",  Nuclear.Model(10.); 
            grid=JAC.Radial.Grid("grid: by given parameters"; rnt = 2.0e-5, h = 5.0e-2, 
                                                              hp = 1.3e-2, NoPoints = 600), 
            initialConfigs=[Configuration("1s 2s^2 2p^6")],
            finalConfigs  =[Configuration("1s^2 2s^2 2p^4"), Configuration("1s^2 2s 2p^5"), 
                            Configuration("1s^2 2p^6")], process = JAC.Auger,
            processSettings = autoSettings )
perform(wb)

Define a radial grid of type MeshGL with 610 grid points
 [rnt=2.000e-05, h=5.000e-02, hp=1.300e-02, NoPoints=600, r_max=4.699e+00;
  B-splines with break points at every 7th point, nsL=86, nsS=87, orderL=7, orderS=8, orderGL=7] 

... in perform['computation: SCF'] ...
perform-aa: Configuration: 1s_1/2^1 2s_1/2^2 2p_1/2^2 2p_3/2^4 
(Re-) Define a new standard subshell list.
(Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 10.0 with mass = 20.5, radius R = 2.8580081360657426 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      1s_1/2      -5.00667217e+01    -5.00667420e+01    +4.04941963e-07    
      2   

  1s_1/2::  en [a.u.] = -3.3912109e+01;   self-cons'cy = 8.2690e-04  [1.9182e-02 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -2.1200222e+00;   self-cons'cy = 4.9473e-03  [1.9182e-02 for sym-block kappa = -1]
  2p_1/2::  en [a.u.] = -1.3028353e+00;   self-cons'cy = 4.5572e-03  [1.2133e-02 for sym-block kappa = 1]
  2p_3/2::  en [a.u.] = -1.2934645e+00;   self-cons'cy = 7.9358e-04  [1.5499e-03 for sym-block kappa = -2]

Iteration 8 for symmetries ... 
  1s_1/2::  en [a.u.] = -3.3898224e+01;   self-cons'cy = 2.0477e-04  [4.4259e-03 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -2.1148729e+00;   self-cons'cy = 1.2159e-03  [4.4259e-03 for sym-block kappa = -1]
  2p_1/2::  en [a.u.] = -1.3015001e+00;   self-cons'cy = 5.1268e-04  [1.4355e-03 for sym-block kappa = 1]
  2p_3/2::  en [a.u.] = -1.2958552e+00;   self-cons'cy = 9.2330e-04  [2.2554e-03 for sym-block kappa = -2]

Iteration 9 for symmetries ... 
  1s_1/2::  en [a.u.] = -3.3898009e+01;   self-cons'cy = 3.1688e-06  [2.3103e-04

  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      2p_3/2      -1.25041630e+01    -1.25041630e+01    +2.96623858e-13    
      2      3p_3/2      -5.55801526e+00    -5.55802266e+00    +1.33227036e-06    
      3      4p_3/2      -3.11045584e+00    -3.12630099e+00    +5.09415692e-03    
      4      5p_3/2      -1.59409282e+00    -2.00074602e+00    +2.55100077e-01    
      5      6p_3/2      +3.55861126e-01    -1.38935143e+00    +4.90419557e+00    
      6      7p_3/2      +2.91974782e+00    -1.02071331e+00    +1.34958954e+00    
      7      8p_3/2      +6.05438849e+00    -7.81461386e-01    +1.12907355e+00    
      8      9p_3/2      +9.73466792e+00    -6.17436217e-01    +1.06342653e+00    
      :       :    
     81     82p_3/2      +9.86809694e+06    -7.43628709e-03    +1.00000000

(Re-) Define a storage array for various B-spline matrices:
*** wx-norm = 0.9999999999999998
Continuum B-spline-Galerkin orbital for energy=2.9792e+01,  kappa=-3 [mpt=410, r[mtp]=2.2779e+00, smallest eigenvalue=2.6139e-02].
Radial potential with effective charge Zbar=2.0002e+00 (Delta-Zbar=0.0000e+00).
Coulomb-sine normalized continuum orbital (on P'/P) with normalization constant N=-1.7489e+00 (Delta-N=1.7983e-02) and phase phi=1.6952e+00 (Delta-N=9.5601e-03).
Compute (Coulomb) Auger matrix of dimension 2 x 1 in the continuum- and initial-state bases for the transition [1- ...] and for partial wave d_5/2 ... done. 
(Re-) Define a storage array for various B-spline matrices:
*** wx-norm = 0.9999999999999993
Continuum B-spline-Galerkin orbital for energy=2.9792e+01,  kappa=2 [mpt=410, r[mtp]=2.2779e+00, smallest eigenvalue=1.2515e-03].
Radial potential with effective charge Zbar=2.0002e+00 (Delta-Zbar=0.0000e+00).
**Skip normalization at i = i, r[i] = 2.0506808416843003  A = -1.66731911

(Re-) Define a storage array for various B-spline matrices:
*** wx-norm = 1.0
Continuum B-spline-Galerkin orbital for energy=2.9787e+01,  kappa=-1 [mpt=410, r[mtp]=2.2779e+00, smallest eigenvalue=6.2554e-06].
Radial potential with effective charge Zbar=2.0002e+00 (Delta-Zbar=0.0000e+00).
Coulomb-sine normalized continuum orbital (on P'/P) with normalization constant N=1.7623e+00 (Delta-N=1.7152e-03) and phase phi=1.9935e+00 (Delta-N=1.8901e-03).
Compute (Coulomb) Auger matrix of dimension 3 x 1 in the continuum- and initial-state bases for the transition [1- ...] and for partial wave s_1/2 ... done. 
(Re-) Define a storage array for various B-spline matrices:
*** wx-norm = 1.0000000000000002
Continuum B-spline-Galerkin orbital for energy=2.9667e+01,  kappa=-3 [mpt=410, r[mtp]=2.2779e+00, smallest eigenvalue=2.7054e-02].
Radial potential with effective charge Zbar=2.0002e+00 (Delta-Zbar=0.0000e+00).
Coulomb-sine normalized continuum orbital (on P'/P) with normalization constant N=1.7348

(Re-) Define a storage array for various B-spline matrices:
*** wx-norm = 0.9999999999999991
Continuum B-spline-Galerkin orbital for energy=2.8813e+01,  kappa=-2 [mpt=410, r[mtp]=2.2779e+00, smallest eigenvalue=8.2299e-08].
Radial potential with effective charge Zbar=2.0002e+00 (Delta-Zbar=0.0000e+00).
Coulomb-sine normalized continuum orbital (on P'/P) with normalization constant N=1.7634e+00 (Delta-N=6.6227e-04) and phase phi=1.2738e+00 (Delta-N=8.6330e-04).
Compute (Coulomb) Auger matrix of dimension 2 x 1 in the continuum- and initial-state bases for the transition [1- ...] and for partial wave p_3/2 ... done. 
(Re-) Define a storage array for various B-spline matrices:
*** wx-norm = 1.0000000000000002
Continuum B-spline-Galerkin orbital for energy=2.8813e+01,  kappa=1 [mpt=410, r[mtp]=2.2779e+00, smallest eigenvalue=3.4946e-10].
Radial potential with effective charge Zbar=2.0002e+00 (Delta-Zbar=0.0000e+00).
Coulomb-sine normalized continuum orbital (on P'/P) with normalization con

In the setting above, we could have specified also a computation for just a few selected transitions or just for transitions within a given energy interval. We shall leave the discussion of these *options* to one of the next tutorials.